# TOPP-HATT

In some cases, we want to optimise an encoding without changing the underlying tree structure. For instance, we may want to use exactly the tree output from the [Bonsai Algorithm](https://ferrmion.readthedocs.io/en/latest/notebooks/bonsai.html).

`ferrmion` lets you do this with the `topphatt` algorithm.

Currently there are two slightly different implementations, 
- `ferrmion.optimize.topphatt`
- `ferrmion.core`

For more details, see our [preprint](https://arxiv.org/abs/2512.13580).

In [ ]:
from ferrmion import core
from ferrmion.utils import fermionic_to_sparse_majorana
from ferrmion.core import anneal_enumerations, encode
from ferrmion.optimize import topphatt
from ferrmion.encode import TernaryTree

import numpy.random as rng
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from pathlib import Path
from copy import deepcopy
import json

We'll start as always by loading in some one and two electron integrals.

In [ ]:
# from openfermion.chem.pubchem import geometry_from_pubchem
# from openfermion.chem.molecular_data import MolecularData
# from openfermion.chem.molecular_data import spinorb_from_spatial
# import openfermionpyscf

# geometry=geometry_from_pubchem("H2O")
# mol = MolecularData(geometry=geometry, basis="6-31g", multiplicity=1)
# mol = openfermionpyscf.run_pyscf(molecule=mol, run_scf=True)
# ones, twos = spinorb_from_spatial(mol.one_body_integrals, mol.two_body_integrals)

# data = {}
# data["e_hf"] = mol.hf_energy
# data["ones"] = ones.tolist()
# data["twos"] = twos.tolist()
# data["constant_energy"] = mol.nuclear_repulsion
# data["geometry"] = geometry
# data["basis"] = "6-31g"
# folder = Path.cwd().joinpath(Path("../../../python/tests/data/"))

# with open(folder.joinpath("h2o_6-31g.json"), 'w+') as file:
#     json.dump(data, file)

In [ ]:

folder = Path.cwd().joinpath(Path("../../../python/tests/data/"))

with open(folder.joinpath("h2o_sto-3g.json"), 'r') as file:
    data = json.load(file)

ones = np.array(data["ones"])
twos = np.array(data["twos"])
enuc = data["constant_energy"]

In [ ]:
def pauli_weights(pauli_hamiltonian: dict[str, float]) -> list[float]:
    unscaled_terms = []
    scaled_terms = []
    for k, v in pauli_hamiltonian.items():
        assert v != 0
        pw = (len(k) - k.count("I")) 
        unscaled_terms.append(pw)
        scaled_terms.append(pw* np.abs(v))

    return (int(np.sum(unscaled_terms)), float(np.sum(scaled_terms)), len(pauli_hamiltonian))

In [ ]:
from ferrmion.core import flatpack_symplectic_matrix
from ferrmion.encode.ternary_tree import TTFlatpack
from os import name
def get_naive_result(tree:TernaryTree, ones, twos, enuc):
    print("Getting Naive result...")
    sdmeans_naive = {}
    ipow, sym = tree._build_symplectic_matrix()
    result = pauli_weights(encode(ipow, sym, ["+-","++--"], [ones, twos], enuc))
    print("Naive result:", result)

    sdmeans_naive = {}
    sdmeans_naive["unscaled"] = result[0]
    sdmeans_naive["scaled"] = result[1]
    sdmeans_naive["length"] = result[2]
    return sdmeans_naive

def get_permuation_results(flatpack, ones, twos, enuc, limit):
    print(f"\nGetting {limit} random results...")
    sdmeans = {}
    sdmeans = {"unscaled":[], "scaled":[], "length":[]}
    ipow, sym = flatpack_symplectic_matrix(flatpack)
    rng = np.random.default_rng(1017)
    
    for i in range(limit):
        if i % 100 == 0:
            print(i)
        perm = rng.permutation(ones.shape[0])
        perm_index = np.ravel(np.column_stack((2*perm, 2*perm +1)))
        result = pauli_weights(encode(ipow[perm_index], sym[perm_index], ["+-", "++--"], [ones, twos], enuc))
        sdmeans["unscaled"].append(result[0]) 
        sdmeans["scaled"].append(result[1])
        sdmeans["length"].append(result[2]) 
        
    print("Permutation Results")

    return sdmeans

def get_annealed_result(flatpack, ones, twos, enuc, coef_weight):
    print("\nGetting Annealed result...")
    sdmeans_annealed = {}
    sdmeans_annealed = {"unscaled":[], "scaled":[], "length":[]}
    ipow, sym = flatpack_symplectic_matrix(flatpack)
    print("Beginning Annealing")
    anneal_ipow, anneal_sym = anneal_enumerations(ipowers=ipow, symplectics=sym, signatures=["+-","++--"], coeffs=[ones, twos], temperature=ones.shape[0], initial_guess=np.array([*range(ones.shape[0])],dtype=np.uint), coefficient_weighted=coef_weight)
    print("Finished annealing.")
    anneal_ham = encode(anneal_ipow, anneal_sym, ["+-","++--"], [ones, twos], enuc)
    result = pauli_weights(anneal_ham)
    print("Annealed result:", result)
    sdmeans_annealed["unscaled"].append(result[0])
    sdmeans_annealed["scaled"].append(result[1])
    sdmeans_annealed["length"].append(result[2])
    return sdmeans_annealed

def get_rust_topphatt_result(flatpack, ones, twos, enuc):
    print("\nRunning rust TOPP-HATT...")
    n_modes = ones.shape[0]
    ipow,sym = core.topphatt(flatpack, n_qubits=n_modes,signatures=["+-","++--"], coeffs=[ones, twos])
        
    print("\nGetting rust TOPP-HATT result...")
    result = pauli_weights(encode(ipow, sym, ["+-","++--"],[ones,twos],enuc))
    print("TOPP-HATT result:", result)
    
    sdmeans_topphatt = {}
    sdmeans_topphatt["unscaled"] = result[0]
    sdmeans_topphatt["scaled"] = result[1]
    sdmeans_topphatt["length"] = result[2]
    return sdmeans_topphatt

def get_topp_hatt_results(tree: TernaryTree, name, ones, twos, enuc, limit=1000):
    flatpack = tree.flatpack()

    initial_ones = deepcopy(ones)
    initial_twos = deepcopy(twos)

    sdmeans_naive = get_naive_result(tree, ones, twos, enuc)
    sdmeans = get_permuation_results(flatpack, ones, twos, enuc, limit)

    sdmeans_annealed_pw = get_annealed_result(flatpack, ones, twos, enuc, False)
    sdmeans_annealed_cpw = get_annealed_result(flatpack, ones, twos, enuc, True)

    sdmeans_topphatt = get_rust_topphatt_result(flatpack, ones, twos, enuc) 

    color_set = {"jw":"tab:blue", "pe":"tab:orange", "bk":"tab:green", "jkmn":"tab:red", "bonsai":"tab:purple", "huffman":"tab:brown", "hatt":"tab:pink"}
    fig, ax = plt.subplots(layout="constrained")

    ax.scatter(sdmeans["unscaled"], sdmeans["scaled"], label="Randomised Permutations", marker=".", color="tab:grey", alpha=0.7)
    ax.plot(sdmeans_naive["unscaled"], sdmeans_naive["scaled"], marker="D", color="tab:red", mec="k", markersize=8)
    ax.plot(sdmeans_annealed_pw["unscaled"], sdmeans_annealed_pw["scaled"], marker="P", color="tab:orange", mec="k", markersize=8)
    ax.plot(sdmeans_annealed_cpw["unscaled"], sdmeans_annealed_cpw["scaled"], marker="X", color="tab:orange", mec="k", markersize=8)
    ax.plot(sdmeans_topphatt["unscaled"], sdmeans_topphatt["scaled"], marker="P", color="tab:green", mec="k", markersize=8)
    # ax.plot(sdmeans_topphatt_py["unscaled"], sdmeans_topphatt_py["scaled"], marker="X", color="tab:blue", mec="k", markersize=8)

    # where some data has already been plotted to ax
    handles, labels = plt.gca().get_legend_handles_labels()
    # manually define a new patch 
    naive_legend = mlines.Line2D([], [], label='Naive Enumeration', marker="D", linestyle="", color="tab:red", mec="k", markersize=8)
    annealed_pw_legend = mlines.Line2D([], [], label='$W_{P}$ Annealed Best', marker="P", linestyle="",color="tab:orange", mec="k", markersize=8)
    annealed_cpw_legend = mlines.Line2D([], [], label='$W_{CP}$ Annealed Best', marker="X", linestyle="",color="tab:orange", mec="k", markersize=8)
    topphatt_legend = mlines.Line2D([], [], label='TOPP-HATT-Rust', marker="P", linestyle="",color="tab:green", mec="k", markersize=8)

    assert np.all(initial_ones == ones)
    assert np.all(initial_twos == twos)

    data = {"random":sdmeans, "naive":sdmeans_naive, "annealed_pw":sdmeans_annealed_pw, "annealed_cpw":sdmeans_annealed_cpw, "topphatt":sdmeans_topphatt}
    with open(f"{name}.json", "w+") as f:
        json.dump(data, f)

    # handles is a list, so append manual patch
    handles.extend([
                    naive_legend,
                    annealed_pw_legend, 
                    annealed_cpw_legend, 
                    topphatt_legend,
                    ])

    ax.set_xlabel("unscaled pauli-weight")
    ax.set_ylabel("coefficient scaled pauli-weight")
    ax.set_title(f"{name} Encodings of $H_2 O$:$STO-3G$")
    fig.legend(handles=handles, loc="outside lower center", ncols=len(handles)//2)
    fig.show()



# Standard Encodings

In [ ]:
from ferrmion.encode.ternary_tree import JordanWigner
get_topp_hatt_results(JordanWigner(ones.shape[0]),"Jordan-Wigner", ones, twos, enuc)

In [ ]:
from ferrmion.encode.ternary_tree import ParityEncoding
get_topp_hatt_results(ParityEncoding(ones.shape[0]),"Parity", ones, twos, enuc)


In [ ]:
from ferrmion.encode.ternary_tree import BravyiKitaev
get_topp_hatt_results(BravyiKitaev(ones.shape[0]),"Bravyi-Kitaev", ones, twos, enuc)

In [ ]:
from ferrmion.encode.ternary_tree import JKMN
get_topp_hatt_results(JKMN(ones.shape[0]),"JKMN", ones, twos, enuc)


# Hamiltonian Optimized Trees

TOPP-HATT can be applied to any ternary tree,  including those which are derived from other optimisation methods.

In [ ]:
from ferrmion.optimize import huffman_ternary_tree

huffman_tree = huffman_ternary_tree(ones, twos)

In [ ]:
huffman_topphatt = get_topp_hatt_results(huffman_tree, name="Huffman", ones=ones,twos=twos,enuc=enuc)

In [ ]:
from ferrmion.visualise import draw_tt
draw_tt(huffman_tree)

In [ ]:
from ferrmion.optimize.hatt import hamiltonian_adaptive_ternary_tree
sparse_ham = fermionic_to_sparse_majorana([(ones, "+-"), (twos,"++--")])
hatt_tree = hamiltonian_adaptive_ternary_tree(deepcopy(sparse_ham), ones.shape[0])
get_topp_hatt_results(hatt_tree, "HATT", ones=ones, twos=twos, enuc=enuc)

In [ ]:
draw_tt(hatt_tree), hatt_tree.enumeration_scheme

# Device Optimized Trees

In [ ]:
import rustworkx as rx
from rustworkx import visualization as rxv
import matplotlib.pyplot as plt
import numpy as np
def heavy_hex_graph():
    graph = rx.PyGraph()
    graph.add_nodes_from(range(ones.shape[0]))
    graph.add_edges_from_no_data(
        [(a,b) for a, b in [
            (0, 1),
            (0, 2),
            (0, 3),
            (1, 4),
            (2, 5),
            (3, 6),
            (4, 7),
            (4, 8),
            (5, 9),
            (5, 10),
            (6, 11),
            (6, 12),
            (7, 13),
            (8, 14),
            (9, 15),
            (10, 16),
            (11, 17),
            (12, 18),
            (13, 19),
            (13, 20),
            (14, 21),
            (14, 22),
            (15, 23),
            (15, 24),
            (16, 25),
            (16, 26),
            (17, 27),
            (17, 28),
            (18, 29),
            (18, 30),
            (22, 31),
            (26, 32),
            (30, 33),
            (31, 34),
            (32, 35),
            (33, 36),
        ] if a<ones.shape[0] and b <ones.shape[1]]
    )
    return graph

In [ ]:
graph = heavy_hex_graph()
rxv.mpl_draw(graph, with_labels=True, node_color="orange", node_size=500)

In [ ]:
from ferrmion.visualise import draw_tt
from ferrmion.optimize import bonsai_algorithm
bonsai= bonsai_algorithm(graph=graph, homogenous=False)
draw_tt(bonsai, type="spaced", enumeration_scheme=bonsai.default_enumeration_scheme())
get_topp_hatt_results(bonsai, name="Bonsai (Heterogeneous, Heavy-Hex)", ones=ones, twos=twos, enuc=enuc)

In [ ]:
from ferrmion.optimize import bonsai_algorithm
bonsai= bonsai_algorithm(graph=graph, homogenous=True)
draw_tt(bonsai, type="spaced")
get_topp_hatt_results(bonsai, name="Bonsai (Homogeneous, Heavy-Hex)", ones=ones, twos=twos,  enuc=enuc)

# Scaling

In [ ]:
timings = []


In [ ]:
from ferrmion import FermionHamiltonian, molecular_hamiltonian
from ferrmion.encode.ternary_tree import JordanWigner, BravyiKitaev, ParityEncoding, JKMN
from ferrmion.core import fermionic_to_sparse_majorana
from openfermion.chem.pubchem import geometry_from_pubchem
from openfermion.chem.molecular_data import MolecularData
from openfermion.chem.molecular_data import spinorb_from_spatial
import openfermionpyscf
import openfermion
import timeit

repeats = 10
for molname in ["H2","LiH","H2O","BH3","CH4","N2","HCN","C2H2","CH3F","ethene","ozone"]:
    geometry=geometry_from_pubchem(molname)
    print(f"{molname=}")
    for basis in ["sto-3g","6-31g"]:
        print(f"{basis=}")
        mol = MolecularData(geometry=geometry, basis=basis, multiplicity=1)
        mol = openfermionpyscf.run_pyscf(molecule=mol, run_scf=True)
        ones, twos = spinorb_from_spatial(mol.one_body_integrals, mol.two_body_integrals)
        n_modes=ones.shape[0]
        fham = molecular_hamiltonian(ones, twos)
        sigs, coeffs = fham.signatures_and_coefficients
        n_terms = len(fermionic_to_sparse_majorana(sigs, coeffs, 0))
        naive_weights = pauli_weights(tree.encode(fham))
        for encoding in [JordanWigner, ParityEncoding, BravyiKitaev, JKMN]:
            print("Encoding: ", encoding)
            tree: TernaryTree = encoding(n_modes=n_modes)
            flatpack = tree.flatpack()
            execution_time = timeit.repeat("core.topphatt(flatpack, n_modes, *fham.signatures_and_coefficients)", globals=globals(), number=1,repeat=repeats)
            print(execution_time)
            topphatt_weights = pauli_weights(tree.encode_topphatt(fham))
            timings.append({"molname":molname, "basis":basis, "encoding":encoding.__name__, "time": execution_time, "n_modes":n_modes,"n_terms":n_terms, "naive_weights":naive_weights, "topphatt_weights":topphatt_weights})
        del(fham)
        del(ones)
        del(twos)
        print()


In [ ]:
with open("timings.json", "w+") as f:
    json.dump(timings, f)

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

for name in ["JordanWigner", "ParityEncoding", "BravyiKitaev", "JKMN"]:
    mt = [(t["n_terms"], np.mean(t["time"])) for t in timings if name in t["encoding"]]
    mt.sort(key=lambda x: x[0])
    modes = np.array([v[0] for v in mt])
    times = np.array([v[1] for v in mt])
    xdata, ydata = modes, times
    plt.scatter(xdata,ydata, marker="x", label=name)
    if name == "JKMN":
        def func(x,a,b): return 10**(a*np.log10(x)+b)
        popt, pcov = curve_fit(func, xdata, ydata)
        print(f"Fit {popt, pcov}")
        plt.plot(xdata, func(xdata, *popt), 'r-', label=f"JKMN fit: {10**popt[1]:.4e}x^{popt[0]:.4f}", color="tab:red")

plt.title("Runtime of TOPP-HATT (Terms)")
plt.yscale("log")
plt.xscale("log")
plt.ylabel("runtime (s)")
plt.xlabel("|$H_{\gamma}$|")
plt.legend()

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

for name in ["JordanWigner", "ParityEncoding", "BravyiKitaev", "JKMN"]:
    mt = [(t["n_modes"], np.mean(t["time"])) for t in timings if name in t["encoding"]]
    mt.sort(key=lambda x: x[0])
    modes = np.array([v[0] for v in mt])
    times = np.array([v[1] for v in mt])
    xdata, ydata = modes, times
    plt.scatter(xdata,ydata, marker="x", label=name)
    # if name == "JKMN":
    #     def func(x,a,b): return 10**(a*np.log10(x)+b)
    #     popt, pcov = curve_fit(func, xdata, ydata)
    #     print(f"Fit {popt, pcov}")
    #     plt.plot(xdata, func(xdata, *popt), 'r-', label=f"JKMN fit: {10**popt[1]:.4e}x^{popt[0]:.4f}", color="tab:red")

plt.title("Runtime of TOPP-HATT (Modes)")
plt.yscale("log")
plt.ylabel("runtime (s)")
plt.xlabel("Number of Modes")
plt.legend()

In [ ]:
with open("timings1.json", "r+") as f:
    timings = json.load(f)

# Comparison of Reductions

In [ ]:
from ferrmion import FermionHamiltonian, molecular_hamiltonian
from ferrmion.encode.ternary_tree import JordanWigner, BravyiKitaev, ParityEncoding, JKMN
from ferrmion.core import fermionic_to_sparse_majorana
from openfermion.chem.pubchem import geometry_from_pubchem
from openfermion.chem.molecular_data import MolecularData
from openfermion.chem.molecular_data import spinorb_from_spatial
import openfermionpyscf
import openfermion
import timeit

reductions = []

for molname in ["H2","LiH","H2O","BH3","CH4","N2","HCN","C2H2","CH3F","ethene","ozone"]:
    geometry=geometry_from_pubchem(molname)
    print(f"{molname=}")
    for basis in ["sto-3g","6-31g", "cc-pVDZ"]:
        mol = MolecularData(geometry=geometry, basis=basis, multiplicity=1)
        mol = openfermionpyscf.run_pyscf(molecule=mol, run_scf=True)
        ones, twos = spinorb_from_spatial(mol.one_body_integrals, mol.two_body_integrals)
        n_modes=ones.shape[0]
        print(f"{n_modes=}")
        fham = molecular_hamiltonian(ones, twos)
        sigs, coeffs = fham.signatures_and_coefficients
        n_terms = len(fermionic_to_sparse_majorana(sigs, coeffs, 0))
        for encoding in [JordanWigner, ParityEncoding, BravyiKitaev, JKMN]:
            tree: TernaryTree = encoding(n_modes=n_modes)
            naive_weights = pauli_weights(tree.encode(fham))
            print("Encoding: ", encoding)
            topphatt_weights = pauli_weights(tree.encode_topphatt(fham))
            reductions.append({"molname":molname, "basis":basis, "encoding":encoding.__name__, "n_modes":n_modes,"n_terms":n_terms, "naive_weights":naive_weights, "topphatt_weights":topphatt_weights})
        del(fham)
        del(ones)
        del(twos)
        print()


In [ ]:
with open("reductions.json", "w+") as f:
    json.dump(reductions, f)

In [ ]:
reductions.sort(key=lambda x: x["n_modes"])
reductions

In [ ]:
import matplotlib.pyplot as plt
colors = {"JordanWigner":"tab:blue", "ParityEncoding":"tab:orange", "BravyiKitaev":"tab:green", "JKMN":"tab:red"}
markers = {"sto-3g": "x", "6-31g":"+"}
for name in ["JordanWigner", "ParityEncoding", "BravyiKitaev", "JKMN"]:
    for basis in ["sto-3g", "6-31g"]:
        this_encoding = [t for t in reductions if name in t["encoding"] and t["basis"] == basis]
        modes = np.array([t["n_modes"] for t in this_encoding])
        pw = np.array([1-float(t["topphatt_weights"][0])/float(t["naive_weights"][0]) for t in this_encoding])
        cpw = np.array([1-float(t["topphatt_weights"][1])/float(t["naive_weights"][1]) for t in this_encoding])
        plt.scatter(pw, cpw, label=name, marker=markers[basis], color = colors[name])
plt.xlabel("$W_{P}$ Reduction")
plt.ylabel("$W_{CP}$ Reduction")
plt.legend()
plt.show()